# Point spread function using Tiny Tim/Galfit

In [1]:
import psf.make_psf as tinytim
import  numpy               as     np
from    astropy.table       import Table
from    astropy.io          import fits
from    scipy.ndimage       import rotate
#from    astropy.cosmology   import Planck18
#import  astropy.units       as     u
import  matplotlib.pyplot   as     plt
import  matplotlib.colors   as     colors  
from    matplotlib          import use
from    tqdm                import tqdm
from concurrent.futures     import ThreadPoolExecutor, as_completed
import sys, os
from astroquery.mast import Observations
from IPython.display import clear_output


def file_name(obj,prefix,filetype='fits'):
    field = obj['subfield'].lower()
    return f"hlsp_clear_hst_wfc3_{field}-{obj['id']}_g102-g141_v4_{prefix}.{filetype}"

def find_data(name,hdu):
    for image in hdu:
        if name == image.name:
            return image

obj_lis = Table.read('obj_lis_selected.fits')



###  gen imput parameters for tinytim

In [28]:
def gen_psf_input_param(obj):
    #try:
    #colnames for param table
    col_names = ['identifier', 'rootname',  'pad', 'filter', 'pa' ,'focus', 'coord_ha', 'coord_hb', 'wavelen_ha', 'wavelen_hb','DELTATIM']
    beam_path = f"data_products/{file_name(obj,'beams')}"
    with fits.open(beam_path) as hdu:
        rows = []
        for image in hdu:
            if image.name == 'SCI':
                #rootname, pad, shape filter
                rootname = image.header['ROOTNAME']
                identifier = f"{obj['subfield']}_{obj['id']}_{image.header['ROOTNAME']}"
                pad        = image.header.get('PAD',0)
                shape      = np.array(image.data.shape)
                filter     = image.header['filter']

                #thumbnail rel. pos
                x0_crop = image.header['ORIGINX']
                y0_crop = image.header['ORIGINY']

                # this part need actual pixel coord for each Ha and Hb wavelen
                coord_ha =  np.array((x0_crop,y0_crop)) - pad  #shape/2 +
                coord_hb =  np.array((x0_crop,y0_crop)) - pad  #shape/2 

                # this still needs focus information
                focus = 0
                #positon angle
                pa = image.header['ORIENTAT']

                #generate spectrum for halpha
                wavelen_ha = 656.28*(1+obj['z_MAP'])
                if wavelen_ha>1700: wavelen_ha = 1700

                #generate spectrum for hbeta
                wavelen_hb = 486.13*(1+obj['z_MAP'])
                if wavelen_hb < 900:wavelen_hb = 900

                #integrationtime
                deltatim = image.header['DELTATIM']

                rows.append((identifier, rootname,  pad, filter, pa ,focus, coord_ha, coord_hb, wavelen_ha, wavelen_hb,deltatim))
        
        #save tiny tim & drizzle input param table
        Table(rows=rows,names=col_names).write(f"psf/obj_param/{obj['subfield']}_{obj['id']}_psf.fits",overwrite=True)
        return f"{obj['subfield']}_{obj['id']} processed"
        
    #except Exception as e:
    #    return f"!error {obj['subfield']}_{obj['id']}: {e}"


from concurrent.futures     import ThreadPoolExecutor, as_completed
def cat_process_gen_psf_input_param(obj_lis,max_threads=1):
    #make directories:
    os.makedirs('psf/obj_param/',exist_ok=True)
    results = []
    if max_threads>1:
        with ThreadPoolExecutor(max_threads) as executor:                                                       
            futures = {executor.submit(gen_psf_input_param,obj):obj for obj in obj_lis}
            for future in tqdm(as_completed(futures), total=len(obj_lis), desc="Processing"):
                results.append(future.result())
        return results
    else:
        for obj in tqdm(obj_lis):
            results.append(gen_psf_input_param(obj))
        return results


if __name__ == '__main__':
    obj_lis = Table.read('obj_lis_selected.fits')
    results = cat_process_gen_psf_input_param(obj_lis,max_threads=8)
    number = 0
    for result in results:
        if 'error' in result:
            number +=1
            print(result)
    print('total number of obj processed:',len(results))
    print('number of failed obj',number)


Processing: 100%|██████████| 158/158 [01:23<00:00,  1.90it/s]

total number of obj processed: 158
number of failed obj 0


### generate psf by calling tinytim + rotation and crop

In [37]:
def rotate_and_crop(image_path,rotating_angle=0,crop_size=50):
    with fits.open(image_path) as hdu:

        rot = rotate(hdu[0].data, 
                    angle=rotating_angle, 
                    reshape=False, 
                    mode='constant')
        
        center_x, center_y = np.array(hdu[0].data.shape) // 2
        half_crop = crop_size // 2
        cropped_data = rot[
            center_y - half_crop : center_y + half_crop,
            center_x - half_crop : center_x + half_crop]
        image = fits.ImageHDU(data=cropped_data, header=hdu[0].header)
    return image

#using tinytim and rotate to generate single psf
def gen_psf(obj,exist_skip=False):
    #try:

    #path to save individual psf and combined psf
    save_path          = f"psf/individual_psf/{obj['subfield']}_{obj['id']}"
    save_path_combined = f"psf/combined_psf/{obj['subfield']}_{obj['id']}"

    if os.path.exists(save_path) and exist_skip==True: 
        clear_output(wait=True)
        return f"{obj['subfield']}_{obj['id']} skipped"

    #load parameter table
    param_table = Table.read(f"psf/obj_param/{obj['subfield']}_{obj['id']}_psf.fits")
    #use integration time as weight
    int_time = np.array(param_table['DELTATIM'])
    weights = int_time/np.sum(int_time)
    #psf_lis to contain individual psf
    psf_lis = fits.HDUList(); psf_lis.append(fits.PrimaryHDU())
    #_combine to store weighted average
    ha_combine = fits.ImageHDU(data=np.zeros((50,50)));hb_combine = fits.ImageHDU(data=np.zeros((50,50)))
    
    #calculate for each individual beam file:
    for i,row in tqdm(enumerate(param_table)):
        print(row)
        identifier, rootname, pad, filter, pa ,focus, coord_ha, coord_hb, wavelen_ha, wavelen_hb, deltatim = row

#------------------this part generate and rotate individual psf--------------------------------
        #ha
        filename_ha = f"psf/individual_psf/{identifier.split('_')[2]}_ha.fits"
        tinytim.make_subsampled_model_psf(filename=filename_ha,
                            psf_size = 4,
                            filter_name = filter,
                            focus = focus,  
                            psf_position = coord_ha,
                            mono = wavelen_ha,
                            subsampling_factor = 2)
        ha_psf = rotate_and_crop(filename_ha,-pa)
        ha_psf.name= f'{rootname}_ha'
        ha_combine.data += ha_psf.data * weights[i]
        #hb
        filename_hb = f"psf/individual_psf/{identifier.split('_')[2]}_hb.fits"
        tinytim.make_subsampled_model_psf(filename=filename_hb,
                            psf_size = 4,
                            filter_name = filter,
                            focus = focus,  
                            psf_position = coord_hb,
                            mono = wavelen_hb,
                            subsampling_factor = 2)
        hb_psf = rotate_and_crop(filename_hb,-pa)
        hb_psf.name = f'{rootname}_hb'
        hb_combine.data += hb_psf.data * weights[i]
#-----------------------------------------------------------------------------------------------

        #normalization and append individual psf
        ha_psf.data = ha_psf.data/np.sum(ha_psf.data)
        hb_psf.data = hb_psf.data/np.sum(hb_psf.data)
        psf_lis.append(ha_psf);psf_lis.append(hb_psf)
        os.remove(filename_ha);os.remove(filename_hb)

    #save combined & individual psf
    psf_lis.append(ha_combine);psf_lis.append(hb_combine)
    psf_lis.writeto(f'{save_path}_psf.fits',overwrite=True)
    ha_combine.writeto(f'{save_path_combined}_ha.fits',overwrite=True)
    hb_combine.writeto(f'{save_path_combined}_hb.fits',overwrite=True)

    #clear output and return
    clear_output(wait=True)
    return f"{obj['subfield']}_{obj['id']} processed"

    #except Exception as e:
    #    return f"!error {obj['subfield']}_{obj['id']}:{e}"



from concurrent.futures     import ThreadPoolExecutor, as_completed
def cat_process_gen_psf_input_param(obj_lis,max_threads=1):
    #make directories:
    os.makedirs('psf/individual_psf',exist_ok=True)
    os.makedirs('psf/combined_psf',exist_ok=True)  
    results = []
    if max_threads>1:
        with ThreadPoolExecutor(max_threads) as executor:
            futures = {executor.submit(gen_psf,obj):obj for obj in obj_lis}
            for future in tqdm(as_completed(futures), total=len(obj_lis), desc="Processing"):
                results.append(future.result())
        return results
    else:
        for obj in tqdm(obj_lis):
            results.append(gen_psf(obj))
        return results


if __name__ == '__main__':
    obj_lis = Table.read('obj_lis_selected.fits')
    results = cat_process_gen_psf_input_param(obj_lis,max_threads=1)
    number = 0
    for result in results:
        if 'error' in result:
            number +=1
            print(result)
    print('total number of obj processed:',len(results))
    print('number of failed obj',number)

 56%|█████▌    | 88/158 [18:22<59:25, 50.94s/it]  

     identifier      rootname pad filter         pa        focus  coord_ha   coord_hb      wavelen_ha         wavelen_hb      DELTATIM
------------------- --------- --- ------ ----------------- ----- ---------- ---------- ------------------ ------------------ ---------
GS4_26698_icoi1mjxq icoi1mjxq 200   G102 73.14400348377133     0 200 .. 207 200 .. 207 1375.9101196358504 1019.1856927813981 50.000412


     identifier      rootname pad filter         pa        focus  coord_ha   coord_hb      wavelen_ha         wavelen_hb      DELTATIM
------------------- --------- --- ------ ----------------- ----- ---------- ---------- ------------------ ------------------ ---------
GS4_26698_icoi1mk0q icoi1mk0q 200   G102 73.14400347979436     0 200 .. 207 200 .. 207 1375.9101196358504 1019.1856927813981 50.000412


     identifier      rootname pad filter         pa        focus  coord_ha   coord_hb      wavelen_ha         wavelen_hb      DELTATIM
------------------- --------- --- ------ ----------------- ----- ---------- ---------- ------------------ ------------------ ---------
GS4_26698_icoi1mk2q icoi1mk2q 200   G102 73.14405552368852     0 198 .. 209 198 .. 209 1375.9101196358504 1019.1856927813981 50.000412


     identifier      rootname pad filter        pa        focus  coord_ha   coord_hb      wavelen_ha         wavelen_hb      DELTATIM
------------------- --------- --- ------ ---------------- ----- ---------- ---------- ------------------ ------------------ ---------
GS4_26698_icoi1mk6q icoi1mk6q 200   G102 73.1440555302776     0 198 .. 209 198 .. 209 1375.9101196358504 1019.1856927813981 50.000412


...

     identifier      rootname pad filter         pa        focus  coord_ha   coord_hb      wavelen_ha         wavelen_hb      DELTATIM
------------------- --------- --- ------ ----------------- ----- ---------- ---------- ------------------ ------------------ ---------
GS4_26698_icoi1tl1q icoi1tl1q 200   G102 151.1387185095184     0 104 .. 532 104 .. 532 1375.9101196358504 1019.1856927813981 50.000412


     identifier      rootname pad filter         pa        focus  coord_ha   coord_hb      wavelen_ha         wavelen_hb      DELTATIM
------------------- --------- --- ------ ----------------- ----- ---------- ---------- ------------------ ------------------ ---------
GS4_26698_icoi1tl5q icoi1tl5q 200   G102 151.1387185337713     0 104 .. 532 104 .. 532 1375.9101196358504 1019.1856927813981 50.000412


     identifier      rootname pad filter         pa        focus  coord_ha   coord_hb      wavelen_ha         wavelen_hb      DELTATIM
------------------- --------- --- ------ ----------------- ----- ---------- ---------- ------------------ ------------------ ---------
GS4_26698_icoi1tlfq icoi1tlfq 200   G102 151.1387185004747     0 103 .. 535 103 .. 535 1375.9101196358504 1019.1856927813981 50.000412


124it [00:41,  2.98it/s]
 56%|█████▌    | 88/158 [19:03<15:09, 13.00s/it]


KeyboardInterrupt: 

### matching psf + hb convolving

In [4]:
from    photutils.psf       import matching as match
from    astropy.convolution import convolve_fft 

#find specific entry in the hdu
def find_data(name,hdu):
    for i,image in enumerate(hdu):
        if name == image.name:
            return i,image

#generate matching kernel for psf_ha & psf_hb
def gen_kernel(psf_hb,psf_ha):
        window = match.CosineBellWindow(alpha=1.3)
        kernel = fits.ImageHDU(
                data = match.create_matching_kernel(psf_hb.data,psf_ha.data,window=window),
                name = 'PSF_MATCH')
        kernel.data = kernel.data/np.sum(kernel.data)
        return kernel

def save_update(image_to_save,extracted):
        for i,image in enumerate(extracted):
            if image.name == image_to_save.name:
                extracted[i] = image_to_save
                return extracted
        extracted.append(image_to_save)
        extracted.flush()
        return extracted

def match_psf(obj):
    print(obj)
    #path for extracted hdu list and combined psf
    path        = f"data_extracted/{file_name(obj,prefix='extracted')}"
    path_to_psf = f"psf/combined_psf/{obj['subfield']}_{obj['id']}"

    #open and update extracted file, add psf_kernel
    with fits.open(path,mode='update') as extracted:
        psf_ha = fits.open(f'{path_to_psf}_ha.fits')[1]
        psf_hb = fits.open(f'{path_to_psf}_hb.fits')[1]
        kernel = gen_kernel(psf_hb,psf_ha)
        
        #convolve hb linemap
        hb     = find_data('LINE_HB',extracted)[1]
        hb_convolve = fits.ImageHDU(data = convolve_fft(hb.data,kernel.data),
        header = hb.header,name = 'LINE_HB_CONV')

        #cnvolve hb weight map
        hb_wht = find_data('LINEWHT_HB',extracted)[1]
        hb_wht_convolve = fits.ImageHDU(data = convolve_fft(hb_wht.data,kernel.data),
        header = hb_wht.header,name = 'LINEWHT_HB_CONV')

        #also do a background subtraction for the convoled image
        hb_convolve_bg = fits.ImageHDU(data = hb_convolve.data - np.median(hb_convolve.data),
        header = hb.header,name = 'LINE_HB_CONV_BG')

        # either update or add data to HDUlist
        extracted = save_update(kernel          ,extracted)
        extracted = save_update(hb_convolve     ,extracted)
        extracted = save_update(hb_wht_convolve ,extracted)
        extracted = save_update(hb_convolve_bg  ,extracted)

        
        #update
        extracted.flush()
        clear_output(wait=True)
        return f"{obj['subfield']}_{obj['id']} processed"


from concurrent.futures     import ThreadPoolExecutor, as_completed
def cat_process_match_psf(obj_lis,max_threads=1):
    results = []
    if max_threads>1:
        with ThreadPoolExecutor(max_threads) as executor:
            futures = {executor.submit(match_psf,obj):obj for obj in obj_lis}
            for future in tqdm(as_completed(futures), total=len(obj_lis), desc="Processing"):
                results.append(future.result())
        return results
    else:
        for obj in tqdm(obj_lis):
            results.append(match_psf(obj))
        return results

if __name__ == '__main__':
    obj_lis = Table.read('obj_lis_selected.fits')
    results = cat_process_match_psf(obj_lis,max_threads=6)
    number = 0
    for result in results:
        if 'error' in result:
            number +=1
            print(result)
    print('total number of obj processed:',len(results))
    print('number of failed obj',number)

Processing: 100%|██████████| 158/158 [01:00<00:00,  2.61it/s]

total number of obj processed: 158
number of failed obj 0


### The focusing info is still needed for best psf generation

In [18]:
def get_focus_info(image):
    obs_id = image.header['ROOTNAME']
    filt = image.header['DFILTER']
    print(obs_id,filt)
    #obsTable = Observations.query_criteria(obs_id=f'*{obs_id[:-3]}', filters=filt)
    #products = Observations.get_product_list(obsTable)
    #print(obs_id,(products['obs_id'==obs_id]))




obs_id = 'ib3747zyq'; filt = 'F140W'
obsTable = Observations.query_criteria(obs_id=f'*ib3747')
products = Observations.get_product_list(obsTable)
for product in products:
    print(product)
#data_prod = ['FLT']                                 # ['FLC','FLT','DRC','DRZ']                  
#data_type = ['CALWF3']                              # ['CALACS','CALWF3','CALWP2','HAP-SVM']    
#Observations.download_products(products) #productSubGroupDescription=data_prod, project=data_type)#
#Observations.download_products(products, productSubGroupDescription=data_prod, project=data_type)#
#clear_output()

 obsID   obs_collection dataproduct_type   obs_id   description  type                 dataURI                  productType productGroupDescription productSubGroupDescription productDocumentationURL project      prvversion     proposal_id     productFilename     size parent_obsid dataRights calib_level filters
-------- -------------- ---------------- --------- ------------- ---- ---------------------------------------- ----------- ----------------------- -------------------------- ----------------------- ------- ------------------- ----------- ----------------------- ---- ------------ ---------- ----------- -------
23824938            HST            image ib3747aaq DADS FLT_HLET    S mast:HST/product/ib3747aaq_flt_hlet.fits   AUXILIARY                      --                   FLT_HLET                      --  CALWF3 3.7.1 (Oct-18-2023)       11600 ib3747aaq_flt_hlet.fits 8640     26508101     PUBLIC           2   F140W
 obsID   obs_collection dataproduct_type   obs_id   description  ty